## Install necessary packages

In [ ]:
!pip install moviepy pytube
!sudo apt update && sudo apt install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,978 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/univers

In [ ]:
!pip3 install whisperx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Add Podcast video URL's to generate transcripts

In [ ]:
urls=['https://youtu.be/zDmG5P1ZCD0?si=RnKoqV0iUd_hVx7f','https://youtu.be/2-2n8uLnxRI?si=pCvxlrH4TKrxeX82','https://www.youtube.com/watch?v=2q0goTPfm_8']

In [ ]:
# Your code for Task 1 goes here!
from pytube import YouTube
import asyncio
import nest_asyncio
nest_asyncio.apply()
async def download_vid(url):
    yt = await asyncio.to_thread(YouTube, url)
    video = await asyncio.to_thread(yt.streams.get_highest_resolution)
    await asyncio.to_thread(video.download)

async def download_all_videos(urls):
    tasks = []
    for url in urls:
        tasks.append(download_vid(url))
    await asyncio.gather(*tasks)

In [ ]:
asyncio.run(download_all_videos(urls))

## Convert downloaded videos to transcripts for knowledge base

In [ ]:
import whisperx
device = "cuda"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

def transcribe_audio(file_path, output_file):
    # Load audio and pad/trim it to fit 30 seconds
    audio = whisperx.load_audio(file_path)
    result = model.transcribe(audio, batch_size=batch_size)
    print(result["segments"]) # before alignment
    # Create a temporary variable to store the concatenated text
    transcription_text = "\n".join([item['text'] for item in result["segments"]])

    with open(output_file, 'a') as f:
        f.write(transcription_text + "\n")

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:08<00:00, 2.08MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [ ]:
op1='/content/fakerape.txt'
transcribe_audio('/content/Fake Rape & Dowry Case Dating Apps Scams Mens Rights & Laws - Deepika Bhardwaj FO142 Raj Shamani.mp4',op1)

Detected language: hi (0.98) in first 30s of audio...
[{'text': ' टिंडर बंबल की डेटिंग स्कैम्स जो मैं आपको बताती हूँ I did a case in Gurgaon जहांपे एक लेडी जो की एक MNC में काम करती है pretty educated वो और एक लड़का ये दोनों जो हैं एक टीम बना के this girl basically meets guys on Bumble तो वो खेल करती है and stays with the guy नेक्स दे जो पुलिस स्टेशन के लिए जाएग', 'start': 0.009, 'end': 28.114}, {'text': ' जो जैपोर्ट से है, उस गैंग में हाइ कोड के लॉयर्स थे, पुलिस आफिसर्स थे, बहुत सारी लड़कियां थी, जो kind of escorts थी, और कुछ लोग थे, जो अपने आपको as a journalist प्रेटेंड करते थे. So अच्छी दर ऐसे reiki करके, they would build repo of these women with these men who were filthy rich, okay?', 'start': 28.114, 'end': 48.677}, {'text': ' उसके बाद वन्स सेक्ष्यल रेलेशिन्शिब वोस इस्टाबलिश्ट उसके बाद इनको कॉल आदा था हाई कॉर्ट के लॉयर्स से की ये लेडी हमारे पास आई है इसने हमारे को बताया की आपने इसको इमोशनली और सेक्ष्यली एक्सप्लॉइट किया है तर्टी टू क्रो', 'start': 48.677, 'end': 77.483}, {'text': '

In [ ]:
op2='/content/MenRights.txt'
transcribe_audio('/content/Why should Men Suffer Men Rights & Laws in India 🇮🇳 Explained by Deepika Bhardwaj  TRS 198.mp4',op2)

Detected language: en (0.74) in first 30s of audio...
[{'text': " When so much of wrong has been done to you and you're still trying to be good to that person and then the person goes out and takes advantage of that situation, things and just spread so many lies, majority of men do not stand up to that. There is this narrative all over society which says, that no stay in the moment, don't think about the future. But a life decision like your job or a life decision like your marriage, you have to have futuristic conversations.", 'start': 0.009, 'end': 25.742}, {'text': ' A lot of young men and boys are still living in that ancient idea of how a woman or a girl is. While women have progressed a lot.', 'start': 25.742, 'end': 43.251}, {'text': " extremely important episode for all the men out there. We're living in very strange times. Considering the Johnny Depp and Amber Heard case that's made worldwide news, I thought it's extremely crucial to bring on a men's rights activist on the sho

In [ ]:
op3='/content/mostImpTalk.txt'
transcribe_audio('/content/The most important talk for INDIAN MEN  Deepika Bhardwaj x Shwetabh.mp4',op3)

Detected language: hi (0.99) in first 30s of audio...
[{'text': ' अपने बारे में यह इंडिया का... अपने बारे में यह अपने बारे में यह इंडिया का... अपने बारे में यह इंडिया का... अपने बारे में यह इंडिया का... अपने बारे में यह इंडिया का... अपने बारे में यह इंडिया का... अपने बारे में यह इंडिया का... अपने बारे', 'start': 0.009, 'end': 9.821}, {'text': " On the top 10 or 20 jo bhi hai, chal raha hai, jo tumhari bhavakti news hai, chalti rati hai, I don't understand. Toh mera pehla question aap se main yaha se shuru karunga. Before we get into the nitty-gritty of this, tumhari sari details and cases and everything. Mere paar 50,000 questions hai kam se kam. Toh pehla aap se simple se ye cheeze hai, aapko kyu lagta hai? Kyuki aap toh ismein bahut time se involved ho. Ye baat main dekha hai ye video me bhi, ki koi women ke saath koi kuch issue hota hai.", 'start': 11.049, 'end': 32.483}, {'text': ' वो national news होती है अर्राइम गुर्ण में चीक भी देता है अब तो सब चीक देते हैं यह चीज हमारे इधर साइड